In [1]:
from math import e
import random

In [8]:
class Leaderboard:
    def __init__(self):
        self.players = {}
        
    def addPlayer(self, name):
        self.players[name] = 1500

    def getPlayers(self):
        for name in self.players.keys():
            print(f"{name} elo: {self.players[name]}")
    
    def getUpsetMult(self, t1p1Elo, t1p2Elo, t2p1Elo, t2p2Elo, t1Score, t2Score, upsetConstant = 1):
        if t1Score >= t2Score:
            winningElo = (t1p1Elo + t1p2Elo) / 2
            losingElo = (t2p1Elo + t2p2Elo) / 2
        else: 
            winningElo = (t2p1Elo + t2p2Elo) / 2
            losingElo = (t1p1Elo + t1p2Elo) / 2

        return upsetConstant * losingElo / winningElo

    def legacy_LobbyEloMult(self, lobbyElo, playerElo, upset, constant = 0.00005): # legacy version using normal distribution
        # e^-constant(delta^2)
        delta = abs(lobbyElo - playerElo)
        exponent = constant * (delta ** 2)
        if upset:
            return 1- (0.9 * e ** -exponent)
        else:
            return 0.9 * e ** -exponent
    
    def getLobbyEloMult(self, lobbyElo, playerElo, win, constant = 0.002): # new version using sigmoid function
        # 1 / (1 - e^-x)
        if win:
            return 2 / (1 + e ** (constant * (playerElo - lobbyElo)))
        else:
            return 2 / (1 + e ** (-constant * (playerElo - lobbyElo)))
            
    def getScoreDeltaMult(self, t1Score, t2Score, scoreDeltaConstant = 1):
        delta = abs(t1Score - t2Score)
        return scoreDeltaConstant * delta / 21

    def add2pGame(self, t1p1, t1p2, t2p1, t2p2, t1Score, t2Score, winElo = 100, lossElo = -100, report = False):
        t1p1Elo = self.players[t1p1]
        t1p2Elo = self.players[t1p2]
        t2p1Elo = self.players[t2p1]
        t2p2Elo = self.players[t2p2]
        
        upsetMult = self.getUpsetMult(t1p1Elo, t1p2Elo, t2p1Elo, t2p2Elo, t1Score, t2Score)
        scoreDeltaMult = self.getScoreDeltaMult(t1Score, t2Score)
        # print(f"upsetMult = {upsetMult} \nscoreDeltaMult={scoreDeltaMult}")
        lobbyElo = sum([t1p1Elo, t1p2Elo, t2p1Elo, t2p2Elo]) / 4
        
        if t1Score > t2Score: # t1 beat t2
            t1p1Change = winElo * upsetMult * self.getLobbyEloMult(lobbyElo, t1p1Elo, True) * scoreDeltaMult
            self.players[t1p1] += t1p1Change

            t1p2Change = winElo * upsetMult * self.getLobbyEloMult(lobbyElo, t1p2Elo, True) * scoreDeltaMult
            self.players[t1p2] += t1p2Change

            t2p1Change = lossElo * upsetMult * self.getLobbyEloMult(lobbyElo, t2p1Elo, False) * scoreDeltaMult
            self.players[t2p1] += t2p1Change

            t2p2Change = lossElo * upsetMult * self.getLobbyEloMult(lobbyElo, t2p2Elo, False) * scoreDeltaMult
            self.players[t2p2] += t2p2Change
            
        else: # t2 beat t1
            t1p1Change = lossElo * upsetMult * self.getLobbyEloMult(lobbyElo, t1p1Elo, False) * scoreDeltaMult
            self.players[t1p1] += t1p1Change

            t1p2Change = lossElo * upsetMult * self.getLobbyEloMult(lobbyElo, t1p2Elo, False) * scoreDeltaMult
            self.players[t1p2] += t1p2Change

            t2p1Change = winElo * upsetMult * self.getLobbyEloMult(lobbyElo, t2p1Elo, True) * scoreDeltaMult
            self.players[t2p1] += t2p1Change

            t2p2Change = winElo * upsetMult * self.getLobbyEloMult(lobbyElo, t2p2Elo, True) * scoreDeltaMult
            self.players[t2p2] += t2p2Change

        if report:
            print("\n")
            print(f"{t1p1} and {t1p2}  vs. {t2p1} and {t2p2}")
            print(f"{((t1p1Elo + t1p2Elo) / 2):.2f} vs. {((t2p1Elo + t2p2Elo) / 2):.2f}")
            print(f"Average lobby Elo: {lobbyElo:.2f}")
            print(f"Score: {t1Score} - {t2Score}")
            print(f"{t1p1}: \n{t1p1Elo:.2f} --> {self.players[t1p1]:.2f} ({'+' if t1p1Change > 0 else ''}{t1p1Change:.2f}) (LobbyEloMult = {self.getLobbyEloMult(lobbyElo, t1p1Elo, t1Score > t2Score):.2f})") 
            print(f"{t1p2}: \n{t1p2Elo:.2f} --> {self.players[t1p2]:.2f} ({'+' if t1p2Change > 0 else ''}{t1p2Change:.2f}) (LobbyEloMult = {self.getLobbyEloMult(lobbyElo, t1p2Elo, t1Score > t2Score):.2f})")
            print(f"{t2p1}: \n{t2p1Elo:.2f} --> {self.players[t2p1]:.2f} ({'+' if t2p1Change > 0 else ''}{t2p1Change:.2f}) (LobbyEloMult = {self.getLobbyEloMult(lobbyElo, t2p1Elo, t1Score < t2Score):.2f})")
            print(f"{t2p2}: \n{t2p2Elo:.2f} --> {self.players[t2p2]:.2f} ({'+' if t2p2Change > 0 else ''}{t2p2Change:.2f}) (LobbyEloMult = {self.getLobbyEloMult(lobbyElo, t2p2Elo, t1Score < t2Score):.2f})")
            
            print(f"Upset Multiplier: {upsetMult:.2f}")
            print(f"Score delta multiplier: {scoreDeltaMult:.2f}")


names = ["will", "jason", "amaris", "jo", "tony", "nancy", "derek", "dillan", "elex", "cat"]
lboard = Leaderboard()
for name in names:
    lboard.addPlayer(name)

for i in range(1, 1000):
    players = random.sample(names, 4)
    score = [21, 21]
    loser = random.randint(0,1)
    score[loser] = random.randint(0,20)
    lboard.add2pGame(players[0], players[1], players[2], players[3], score[0], score[1], report = False)

for i in range(1, 6):
    players = random.sample(names, 4)
    score = [21, 21]
    loser = random.randint(0,1)
    score[loser] = random.randint(0,20)
    lboard.add2pGame(players[0], players[1], players[2], players[3], score[0], score[1], report = True)




dillan and nancy  vs. tony and jason
1430.09 vs. 1651.85
Average lobby Elo: 1540.97
Score: 5 - 21
dillan: 
1353.33 --> 1299.60 (-53.73) (LobbyEloMult = 0.81)
nancy: 
1506.84 --> 1443.13 (-63.71) (LobbyEloMult = 0.97)
tony: 
1593.09 --> 1655.61 (+62.53) (LobbyEloMult = 0.95)
jason: 
1710.60 --> 1765.48 (+54.88) (LobbyEloMult = 0.83)
Upset Multiplier: 0.87
Score delta multiplier: 0.76


nancy and jo  vs. tony and will
1417.61 vs. 1529.25
Average lobby Elo: 1473.43
Score: 21 - 0
nancy: 
1443.13 --> 1554.27 (+111.14) (LobbyEloMult = 1.03)
jo: 
1392.09 --> 1508.72 (+116.63) (LobbyEloMult = 1.08)
tony: 
1655.61 --> 1528.30 (-127.31) (LobbyEloMult = 1.18)
will: 
1402.89 --> 1302.61 (-100.28) (LobbyEloMult = 0.93)
Upset Multiplier: 1.08
Score delta multiplier: 1.00


jo and dillan  vs. jason and cat
1404.16 vs. 1734.45
Average lobby Elo: 1569.31
Score: 10 - 21
jo: 
1508.72 --> 1468.88 (-39.84) (LobbyEloMult = 0.94)
dillan: 
1299.60 --> 1268.37 (-31.24) (LobbyEloMult = 0.74)
jason: 
1765.48 -

In [5]:
import os
from os.path import exists

exists(os.getcwd()+"/BBC ELO discord bot/bot.py")

True

In [3]:
os.getcwd()+"/BBC ELO discord bot/bot.py"

NameError: name 'os' is not defined

In [209]:
lb = Leaderboard()
with open("lb.pickle", 'wb') as f:
    pickle.dump(lb, f)

In [9]:
import pickle
with open("lb.pickle", "rb") as f:
    lb2 = pickle.load(f)

In [10]:
lb2.addPlayer('test')
lb2.getPlayers()

test elo: 1500
